# Set up the experiment

## Import Required Libraries

In [1]:
import numpy as np
from ThresholdCalculator import ThresholdCalculator
from tensorflow import keras
import tensorflow as tf
from keras.datasets import cifar10
import Evaluation
from AdversarialGenerator import AdversarialGenerator

Using TensorFlow backend.


## Load Dateset

In [ ]:
# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Data normalization
x_train = x_train.astype('float32')
x_test =  x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

## Load Image Classifiers

In [ ]:
# Disable eager execution mode of tensorflow to run Adversarial Attack
tf.compat.v1.disable_eager_execution()

MLP = keras.models.load_model('./Models/MLP.h5')
VGG19 = keras.models.load_model('./Models/VGG19.h5')
ResNet50 = keras.models.load_model('./Models/ResNet50.h5')

## Generate Adversarial Examples

### Prepare Adversarial Generator

In [ ]:
#Number of adversarial examples to be generated
n = 500
examples_path = "./Adversarial_Examples/"
MLP_adv_gen = AdversarialGenerator(n,x_test,examples_path,"MLP",MLP)
VGG19_adv_gen = AdversarialGenerator(n,x_test,examples_path,"VGG19",VGG19)
ResNet50_adv_gen = AdversarialGenerator(n,x_test,examples_path,"ResNet50",ResNet50)


### Generate Adversarial Examples for MLP

### Basic Iterative Method (BIM)

In [ ]:
#MLP_adv_gen.generate_adversarial_examples("BIM")
VGG19_adv_gen.generate_adversarial_examples("BIM")
ResNet50_adv_gen.generate_adversarial_examples("BIM")

### Carlini and Wagner (CW)

In [ ]:
MLP_adv_gen.generate_adversarial_examples("CW")

### DeepFool (DF)

In [ ]:
MLP_adv_gen.generate_adversarial_examples("DF")

### Fast Gradient Sign Method (FGSM)

In [ ]:
# MLP_adv_gen = AdversarialGenerator(n,x_test,examples_path,"MLP",MLP)
# MLP_adv_gen.generate_adversarial_examples("FGSM")

VGG19_adv_gen = AdversarialGenerator(n,x_test,examples_path,"VGG19",VGG19)
VGG19_adv_gen.generate_adversarial_examples("FGSM")

ResNet50_adv_gen = AdversarialGenerator(n,x_test,examples_path,"ResNet50",ResNet50)
ResNet50_adv_gen.generate_adversarial_examples("FGSM")

### PixelAttack (PA)

In [ ]:
MLP_adv_gen.generate_adversarial_examples("PA")

### Square Attack (SA)

In [ ]:
MLP_adv_gen = AdversarialGenerator(n,x_test,examples_path,"MLP",MLP)
MLP_adv_gen.generate_adversarial_examples("SA")

### Spatial Transformations Attack (ST)

In [ ]:
# MLP_adv_gen.generate_adversarial_examples("ST")
# VGG19_adv_gen.generate_adversarial_examples("ST")
ResNet50_adv_gen.generate_adversarial_examples("ST")

### Universal Adversarial Perturbations (UAP) Attack

In [ ]:
# MLP_adv_gen = AdversarialGenerator(n,x_test,examples_path,"MLP",MLP)
MLP_adv_gen.generate_adversarial_examples("UAP")
VGG19_adv_gen.generate_adversarial_examples("UAP")
ResNet50_adv_gen.generate_adversarial_examples("UAP")

In [ ]:
MLP_adv_gen = AdversarialGenerator(n,x_test,examples_path,"MLP",MLP)
MLP_adv_gen.generate_adversarial_examples("UAP")

### Zeroth-Order Optimization (ZOO) Attack

In [ ]:
# MLP_adv_gen = AdversarialGenerator(n,x_test,examples_path,"MLP",MLP)
# MLP_adv_gen.generate_adversarial_examples("ZOO")

# VGG19_adv_gen = AdversarialGenerator(n,x_test,examples_path,"VGG19",VGG19)
# VGG19_adv_gen.generate_adversarial_examples("ZOO")

ResNet50_adv_gen = AdversarialGenerator(n,x_test,examples_path,"ResNet50",ResNet50)
ResNet50_adv_gen.generate_adversarial_examples("ZOO")

# Evaluate The Defense

## Evaluate Threshold Value

In [2]:
Evaluation.test_attacks(threshold=1.7,num_of_runs=1)


FGSM

BIM

UAP

PA

ST

ZOO


In [3]:
x_org, x_adv = Evaluation.load("PA","MLP")
max_threshold, mean_threshold =ThresholdCalculator(x_org,x_adv).calculate_threshold(norm=1)
thresholds = np.linspace(start=max_threshold - mean_threshold, stop=max_threshold + mean_threshold, num=14)
Evaluation.test_thresholds(thresholds=thresholds, num_of_runs=1, norm=1)

In [4]:
max_threshold, mean_threshold =ThresholdCalculator(x_org,x_adv).calculate_threshold(norm=2)
thresholds = np.linspace(start=max_threshold - mean_threshold, stop=max_threshold + mean_threshold, num=14)
Evaluation.test_thresholds(thresholds=thresholds, num_of_runs=1, norm=2)

In [5]:
max_threshold, mean_threshold =ThresholdCalculator(x_org,x_adv).calculate_threshold(norm=3)
thresholds = np.linspace(start=max_threshold - mean_threshold, stop=max_threshold + mean_threshold, num=14)
Evaluation.test_thresholds(thresholds=thresholds, num_of_runs=1, norm=3)

## Evaluate Buffer size


In [2]:
org, adv = Evaluation.load("PA","MLP")
images, labels = Evaluation.combine(org,adv)

In [3]:
images = images.reshape(1000,32*32*3)
buffer_sizes = list(range(200,1050,200))
Evaluation.test_buffer_sizes(buffer_sizes=buffer_sizes, threshold=2.12,som_data=images)

[0.565, 0.823, 1.0, 1.0, 1.0]
[0.997, 0.999, 1.0, 0.998, 0.999]
[0.56, 0.827, 1.0, 1.0, 1.0]
[0.996, 0.998, 0.994, 0.998, 0.998]
[0.551, 0.815, 1.0, 1.0, 1.0]
[0.992, 0.999, 0.999, 0.998, 0.996]
